In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn.functional as F


In [3]:
a = torch.arange(10).reshape(5, 2)

In [6]:
a.split(1, dim=1)

(tensor([[0],
         [2],
         [4],
         [6],
         [8]]),
 tensor([[1],
         [3],
         [5],
         [7],
         [9]]))

In [16]:
emb = 30

x = torch.rand(10,9,emb)
n_head = 3

c_attn = nn.Linear(emb, 3 * emb)

B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

# calculate query, key, values for all heads in batch and move head forward to be the batch dim
q, k ,v  = c_attn(x).split(emb, dim=2)

In [12]:
q.shape 

torch.Size([10, 9, 30])

In [18]:
k = k.view(B, T, n_head, C // n_head).transpose(1, 2) # (B, nh, T, hs)
q = q.view(B, T, n_head, C // n_head).transpose(1, 2) # (B, nh, T, hs)
v = v.view(B, T, n_head, C // n_head).transpose(1, 2) # (B, nh, T, hs)

In [19]:
k.shape

torch.Size([10, 3, 9, 10])

In [56]:
class ImprovedTwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ImprovedTwoLayerNN, self).__init__()
        # Increase depth and capacity
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.LeakyReLU()
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.5)  # Adjust dropout rate as needed
        
        # Additional layer
        self.layer2 = nn.Linear(hidden_size, hidden_size * 2)
        self.relu2 = nn.LeakyReLU()
        self.batch_norm2 = nn.BatchNorm1d(hidden_size * 2)
        self.dropout2 = nn.Dropout(0.5)  # Adjust dropout rate as needed
        
        # Output layer
        self.layer3 = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        
        x = self.layer3(x)
        return x
    
class TwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size,  output_size):
        super(TwoLayerNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

class CustomTransformerModel(nn.Module):
    def __init__(self):
        super(CustomTransformerModel, self).__init__()
        self.embedding = nn.Linear(768, 4096) # Example embedding size
        self.transformer_block = nn.TransformerEncoderLayer(d_model=4096, nhead=8)
        self.output_linear = nn.Linear(4096, 4096)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2) # Transformer expects [seq_len, batch, features]
        x = self.transformer_block(x)
        x = x.permute(1, 0, 2) # Revert permutation
        x = torch.mean(x, dim=1) # Aggregating over sequence
        x = self.output_linear(x)
        return x

In [37]:
def train(data_in, data_out, model, criterion, num_epochs, save=False, learning_rate=0.001):
    
    
    # Create a complete dataset
    full_dataset = TensorDataset(data_in.to("cuda"), data_out.to("cuda"))

    # Define the sizes for your training and validation sets
    total_size = len(full_dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size

    # Split the dataset
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    # Create DataLoaders for both training and validation sets
    train_dataloader = DataLoader(train_dataset, batch_size=400, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=500)  # No need to shuffle the validation data


    model.to("cuda")
     
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)



    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_dataloader:
            inputs, targets = inputs.to("cuda").float(), targets.to("cuda").float()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
         
            running_loss += loss.item() * inputs.size(0)
    
        epoch_loss = running_loss / len(train_dataloader.dataset)
        # Validation phase
        model.eval()  # Set the model to evaluation mode
        running_loss = 0.0
        with torch.no_grad():  # No gradients need to be calculated
            for inputs, targets in val_dataloader:
                inputs, targets = inputs.to("cuda").float(), targets.to("cuda").float()
                outputs = model(inputs)
                
                loss = criterion(outputs, targets)
                running_loss += loss.item() * inputs.size(0)
        val_loss = running_loss / len(val_dataloader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.8f}, Validation Loss: {val_loss:.8f}')
        
    if(save):
        # Ensure the model is in evaluation mode
        model.eval()

        predictions = []
        with torch.no_grad():  # No gradients needed for inference
            for inputs, _ in full_dataset:  # Assuming your dataset returns inputs and targets
                inputs = inputs.to('cuda').float().unsqueeze(0)
                
                # Get the model output
                outputs = model(inputs)
                

                predictions.append(outputs.cpu())

        # Concatenate all batches of predictions
        all_predictions = torch.stack(predictions, dim=0)

        # Save the tensor to a file
        torch.save(all_predictions, 'model_predictions.pth')



In [17]:
id = "1"

loaded_activations  = torch.load(f'activations-{id}.pth')
loaded_embeddings = torch.load(f"embeddings-{id}.pth")
loaded_residual_stream = torch.load(f"residual_stream-{id}.pth")
#loaded_rebased_embeddings = torch.load(f"rebased_embeddings-{id}.pth")
loaded_one_hot = torch.load(f"one-hot-{id}.pth")

In [34]:
# consts
attention = 2
ff = 3

layer = 10


residual_stream =  [i[layer, -5:-1, :] for i in loaded_residual_stream]

residual_stream = torch.stack(residual_stream)


In [55]:
loaded_embeddings.shape

torch.Size([195, 4096])

In [57]:




data_in = residual_stream
data_out = loaded_embeddings


criterion = nn.MSELoss() 


model = CustomTransformerModel().to("cuda")


In [58]:
model(residual_stream[0: 3].to("cuda").float()).shape

torch.Size([3, 4096])

In [62]:
train(data_in, data_out, model, criterion, num_epochs=20)
    

Epoch [1/20], Training Loss: 6.47049284, Validation Loss: 16.50962067
Epoch [2/20], Training Loss: 15.21453571, Validation Loss: 9.84245014
Epoch [3/20], Training Loss: 9.61733818, Validation Loss: 7.43053102
Epoch [4/20], Training Loss: 7.60068083, Validation Loss: 7.92670298
Epoch [5/20], Training Loss: 8.04406452, Validation Loss: 8.66712856
Epoch [6/20], Training Loss: 8.62406158, Validation Loss: 8.16909981
Epoch [7/20], Training Loss: 8.07136154, Validation Loss: 7.18315268
Epoch [8/20], Training Loss: 7.09722900, Validation Loss: 6.87435865
Epoch [9/20], Training Loss: 6.76458263, Validation Loss: 7.22080517
Epoch [10/20], Training Loss: 7.06584454, Validation Loss: 7.31530142
Epoch [11/20], Training Loss: 7.18884516, Validation Loss: 6.94839573
Epoch [12/20], Training Loss: 6.91547871, Validation Loss: 6.55599022
Epoch [13/20], Training Loss: 6.62228251, Validation Loss: 6.45699453
Epoch [14/20], Training Loss: 6.57377911, Validation Loss: 6.55776930
Epoch [15/20], Training Los

In [69]:
# Ensure the model is in evaluation mode
model.eval()

full_dataset = TensorDataset(data_in.to("cuda"), data_out.to("cuda"))

predictions = []
with torch.no_grad():  # No gradients needed for inference
    for inputs, _ in full_dataset:  # Assuming your dataset returns inputs and targets
        inputs = inputs.to('cuda').float().unsqueeze(0)
        
        # Get the model output
        outputs = model(inputs)
        

        predictions.append(outputs.cpu())

# Concatenate all batches of predictions
all_predictions = torch.stack(predictions, dim=0)

# Save the tensor to a file
torch.save(all_predictions, 'model_predictions.pth')

In [70]:
all_predictions 

tensor([[[ 2.1009,  3.0100,  2.5069,  ..., -2.2787, -1.8265, -2.2440]],

        [[ 1.8054,  2.9324,  3.0807,  ..., -2.1298, -1.8219, -2.0258]],

        [[ 1.4284,  2.8934,  2.6290,  ..., -2.0014, -1.6157, -1.9839]],

        ...,

        [[ 1.9224,  3.1655,  2.8131,  ..., -2.1422, -1.9931, -2.2045]],

        [[ 1.8757,  2.5912,  1.9763,  ..., -1.8930, -1.8025, -1.9383]],

        [[ 1.8031,  2.9315,  2.6223,  ..., -2.0391, -1.8449, -2.0276]]])

In [71]:
loaded_one_hot.shape

torch.Size([21, 195])

In [72]:
labels = [
    'adventure',
    'happiness',
    'friendship',
    'magic',
    "space",
    "exploration",
    "dragons",
    "knights",
    "investigation",
    "animals",
    "childhood",
    "disease",
    "princess",
    "dreams",
    "boys",
    "girls",
    "plants",
    "travel",
    "woods",
    "robots",
    "machines"
]


In [73]:
loaded_texts = torch.load(f"generated_texts-{id}.pth")

In [74]:
loaded_one_hot.shape

torch.Size([21, 195])

In [75]:
all_predictions.squeeze()[3]

tensor([ 2.2111,  3.0665,  3.2989,  1.3309, -1.2580, -0.4159,  0.8268, -2.5817,
        -1.4823,  3.0156,  2.9174, -3.9712,  0.1598, -3.6134, -0.8241,  0.3157,
        -1.7431, -1.0077, -2.3642, -1.8939, -2.3697])

In [76]:
pred_labels = [labels[i] for i in torch.argmax(all_predictions.squeeze(), dim=1)]

In [77]:
pred_labels

['childhood',
 'friendship',
 'happiness',
 'friendship',
 'friendship',
 'childhood',
 'happiness',
 'childhood',
 'animals',
 'happiness',
 'happiness',
 'childhood',
 'happiness',
 'happiness',
 'childhood',
 'happiness',
 'friendship',
 'childhood',
 'happiness',
 'childhood',
 'happiness',
 'happiness',
 'childhood',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'childhood',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'happiness',
 'childhood',
 'happiness',
 'happiness',
 'childhood',
 'friendship',
 'happiness',
 'happiness',
 'happiness',
 'friendship',
 'friendship',
 'childhood',
 'happiness',
 'childhood',
 'happiness',
 'friendship',
 'happiness',
 'friendship',
 'friendship',
 'friendship',
 'friendship',
 'friendship',
 'childhood',
 'happiness',
 'happiness',
 'animals',
 'childhood',
 'friendship',
 'animals'

In [56]:
decoded_labels = [labels[i] for i in torch.argmax(loaded_one_hot, dim=0)]


In [21]:
loaded_texts

[' He hopes one day he will find it and make his life easier.\nOne day, a wizard came to visit the dragon. The wizard said, “I can help you find a special cure, but it won’t be easy.”\nThe dragon was excited and asked, “What is it?”\nThe wizard replied, “You will have to wait and see.”\nSo the dragon waited patiently and after a few days, the wizard sent the',
 ' They search for clues and often have to try to solve a mystery.\nOne day, a little rabbit was out exploring and came across a group of ducks. The ducks were quacking and seemed to be looking for something. The rabbit decided to follow them and soon found out why.\nThe group of animals was trying to find a lost puppy. The rabbit was excited and wanted to help, so he started to search.\nThe rabbit was very',
 ' She loved to talk to the plants and feel the sunshine on her face.\nOne day, she noticed a plant that was leaning against a tree. The child leaned down to take a closer look and noticed something strange. It was a little 

In [89]:

for e in [(loaded_texts[i], decoded_labels[i], pred_labels[i]) for i in range(0,40)]:
    t = e[0].replace('\n', '')
    print(f"{t}\nassinged label: {e[1]} predicted label: {e[2]}\n")


 He hopes one day he will find it and make his life easier.One day, a wizard came to visit the dragon. The wizard said, “I can help you find a special cure, but it won’t be easy.”The dragon was excited and asked, “What is it?”The wizard replied, “You will have to wait and see.”So the dragon waited patiently and after a few days, the wizard sent the
assinged label: dragons predicted label: childhood

 They search for clues and often have to try to solve a mystery.One day, a little rabbit was out exploring and came across a group of ducks. The ducks were quacking and seemed to be looking for something. The rabbit decided to follow them and soon found out why.The group of animals was trying to find a lost puppy. The rabbit was excited and wanted to help, so he started to search.The rabbit was very
assinged label: animals predicted label: friendship

 She loved to talk to the plants and feel the sunshine on her face.One day, she noticed a plant that was leaning against a tree. The child le